In [13]:
using   BenchmarkTools
p=2
function pow_array(x::Vector{Float64})
    s=0.0
    for y in x
        s=s+y^p
    end
    return s
    
end
pow_array([1.0,2.0])
t=rand(1000_000)
@btime pow_array($t)

  54.290 ms (3000000 allocations: 45.78 MiB)


333005.05518908106

In [9]:
@code_warntype pow_array(t)


Variables
  #self#::Core.Compiler.Const(pow_array, false)
  x::Array{Float64,1}
  s::Any
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  y::Float64

Body::Any
1 ─       (s = 0.0)
│   %2  = x::Array{Float64,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (y ^ Main.p)::Any
│         (s = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [15]:
const p2=2
function pow_array2(x::Vector{Float64})
    s=0.0
    for y in x
        s=s+y^p2
    end
    return s
    
end
pow_array2([1.0,2.0])
t=rand(1000_000)
@btime pow_array2($t)


  1.176 ms (0 allocations: 0 bytes)


333376.1937827427

In [17]:
@code_warntype pow_array2([3.0,2.0])

Variables
  #self#::Core.Compiler.Const(pow_array2, false)
  x::Array{Float64,1}
  s::Float64
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  y::Float64

Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Array{Float64,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Float64
│   %11 = (y ^ Main.p2)::Float64
│         (s = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [19]:
function f(x)
    a=5x
    b=a+3
    return b
end
g(x)=f(2x)

@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = Base.mul_int(5, %1)::Int64
│   %3 = Base.add_int(%2, 3)::Int64
└──      return %3
) => Int64

In [22]:
@code_llvm g(3)


;  @ In[19]:6 within `g'
; Function Attrs: uwtable
define i64 @julia_g_1945(i64) #0 {
top:
; ┌ @ In[19]:2 within `f'
; │┌ @ int.jl:87 within `*'
    %1 = mul i64 %0, 10
; │└
; │ @ In[19]:3 within `f'
; │┌ @ int.jl:86 within `+'
    %2 = add i64 %1, 3
; └└
  ret i64 %2
}


In [28]:
function f(x)
    a=5x
    b=a+3
    c=a-4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d=c^3
    else
        d=c^2
    end
end


@show @code_typed g(3)

@btime g(3)

#= In[28]:15 =# @code_typed(g(3)) = CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = invoke Main.f(%1::Int64)::Int64
└──      return %2
) => Int64
  0.001 ns (0 allocations: 0 bytes)


676

In [30]:
@inline function f_in(x)
    a=5x
    b=a+3
    c=a-4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d=c^3
    else
        d=c^2
    end
end

g_in(x)=f_in(2x)
@show @code_typed g_in(3)
@btime g_in(3)

#= In[30]:15 =# @code_typed(g_in(3)) = CodeInfo(
1 ─ %1  = Base.mul_int(2, x)::Int64
│   %2  = Base.mul_int(5, %1)::Int64
│   %3  = Base.sub_int(%2, 4)::Int64
│   %4  = Base.slt_int(%3, 0)::Bool
└──       goto #3 if not %4
2 ─       Main.DomainError()::Union{}
└──       $(Expr(:unreachable))::Union{}
3 ┄ %8  = Base.slt_int(%3, 2)::Bool
└──       goto #5 if not %8
4 ─ %10 = Base.mul_int(%3, %3)::Int64
│   %11 = Base.mul_int(%10, %3)::Int64
└──       goto #6
5 ─ %13 = Base.mul_int(%3, %3)::Int64
└──       goto #6
6 ┄ %15 = φ (#4 => %11, #5 => %13)::Int64
└──       return %15
) => Int64
  0.001 ns (0 allocations: 0 bytes)


676

In [2]:
function poly_naive(x,a...)
    p=zero(x)
    for i =1:length(a)
        p=p+a[i]*x^(i-1)
    end
    return p
end

@show peakflops()

peakflops() = 

5.934275011290886e9

5.934275011290886e9


In [8]:
using  BenchmarkTools

f_naive(x)=poly_naive(x,1,2,3,4,5,6,7,8,9)
x=3.5
@show f_naive(1)
@btime f_naive(x)

f_naive(1) = 45
  424.875 ns (3 allocations: 48 bytes)


271125.95703125

In [16]:
function poly_horner(x,a...)
    b=zero(x)
    for i in  length(a):-1:1
        b=a[i]+b*x
    end
    return b
end

f_horner(x)=poly_horner(x,1,2,3,4,5,6,7,8,9)
f_horner(1)
x=3.5
@btime f_horner($x)

  4.199 ns (0 allocations: 0 bytes)


271125.95703125

In [ ]:
function prod_dim(x::Array{T,N} ) where {T,N}
    s=1
    for i in 1:N
        s=s*size(x,i)
    end
    return s
end